In [1]:
from appgeopy import *
from my_packages import *
from signaltools import *

In [2]:
def read_peak_trough_data(file_path, sheet_name, data_type="peaks"):
    """
    Read peak or trough data from an Excel sheet and add a 'type' column.

    Parameters:
    - file_path (str): Path to the Excel file.
    - sheet_name (str): Name of the Excel sheet to read data from.
    - data_type (str): Type of the data ("peaks" or "troughs").

    Returns:
    - pd.DataFrame: DataFrame containing the data with an added 'type' column.
    """
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df["type"] = data_type
    return df

def calculate_date_diff_threshold(df, date_col="dates", quantile=0.25, min_threshold=None):
    """
    Calculate the threshold for date differences between consecutive data points.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the time series data.
    - date_col (str): The column containing the dates.
    - quantile (float): The quantile to calculate for the threshold.
    - min_threshold (float or None): If specified, the threshold will be at least this value.

    Returns:
    - float: The threshold for date differences.
    """
    date_diffs = df[date_col].diff().dropna().abs().dt.days
    threshold = date_diffs.quantile(quantile)
    if min_threshold is not None:
        threshold = max(threshold, min_threshold)
    return threshold

def compare_values_within_threshold(idx_current, idx_neighbor, dates, values, threshold, data_type="peaks"):
    """
    Compare values at two indices if their date difference is within the threshold.

    Parameters:
    - idx_current (int): Index of the current data point.
    - idx_neighbor (int): Index of the neighboring data point.
    - dates (pd.Series): Series of dates.
    - values (pd.Series): Series of values corresponding to dates.
    - threshold (float): Threshold for allowable date difference.
    - data_type (str): Type of data ("peaks" or "troughs").

    Returns:
    - int or None: Index of the data point to remove, or None if no removal.
    """
    date_current = dates.iloc[idx_current]
    date_neighbor = dates.iloc[idx_neighbor]
    value_current = values.iloc[idx_current]
    value_neighbor = values.iloc[idx_neighbor]
    day_diff = abs((date_current - date_neighbor).days)

    if day_diff < threshold:
        if data_type == "peaks":
            # For peaks, remove the lower peak
            return idx_neighbor if value_current > value_neighbor else idx_current
        elif data_type == "troughs":
            # For troughs, remove the higher trough
            return idx_neighbor if value_current < value_neighbor else idx_current
    return None

def identify_redundant_indices(df, threshold, data_type="peaks", date_col="dates", value_col="values"):
    """
    Identify indices of data points to remove based on proximity and value comparison.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the time series data.
    - threshold (float): Threshold for allowable date difference.
    - data_type (str): Type of data ("peaks" or "troughs").
    - date_col (str): Column name for dates.
    - value_col (str): Column name for values.

    Returns:
    - list: List of indices to remove.
    """
    indices_to_remove = []

    dates = df[date_col].reset_index(drop=True)
    values = df[value_col].reset_index(drop=True)

    for i in range(len(df)):
        idx_current = i
        # Compare with previous and next points
        neighbor_indices = []
        if i > 0:
            neighbor_indices.append(i - 1)
        if i < len(df) - 1:
            neighbor_indices.append(i + 1)
        for idx_neighbor in neighbor_indices:
            idx_to_remove = compare_values_within_threshold(
                idx_current=idx_current,
                idx_neighbor=idx_neighbor,
                dates=dates,
                values=values,
                threshold=threshold,
                data_type=data_type
            )
            if idx_to_remove is not None:
                indices_to_remove.append(df.index[idx_to_remove])

    # Remove duplicate indices
    indices_to_remove = list(set(indices_to_remove))
    return indices_to_remove

def filter_consecutive_duplicates(df, value_col="values", type_col="type"):
    """
    Filter DataFrame by keeping the largest peaks and smallest troughs when consecutive
    peaks or troughs are detected.

    Parameters:
    - df (pd.DataFrame): DataFrame containing both peaks and troughs.
    - value_col (str): Column name for values.
    - type_col (str): Column name specifying the type of data ("peaks" or "troughs").

    Returns:
    - pd.DataFrame: Filtered DataFrame with desired peaks and troughs.
    """
    keep_indices = []
    i = 0
    while i < len(df):
        current_type = df.iloc[i][type_col]
        current_value = df.iloc[i][value_col]
        max_min_value = current_value
        max_min_index = i
        j = i
        while j + 1 < len(df) and df.iloc[j + 1][type_col] == current_type:
            j += 1
            next_value = df.iloc[j][value_col]
            if current_type == "peaks":
                if next_value > max_min_value:
                    max_min_value = next_value
                    max_min_index = j
            elif current_type == "troughs":
                if next_value < max_min_value:
                    max_min_value = next_value
                    max_min_index = j
        keep_indices.append(df.index[max_min_index])
        i = j + 1
    return df.loc[keep_indices].reset_index(drop=True)

In [3]:
# Define main parameters (user can modify these as needed)
EXCEL_FOLDER = "temp4_v2"
HDF5_FILE_PATH = "20240903_GWL_CRFP.h5"
THRESHOLD_QUANTILE = 0.2
MIN_THRESHOLD = None  # Minimum threshold for date differences, can be set to None

# Load HDF5 File and Extract Data
with h5py.File(HDF5_FILE_PATH, "r") as hdf5_file:
    existing_data_dict = h5pytools.hdf5_to_data_dict(hdf5_file)
    available_datasets = h5pytools.list_datasets(hdf5_file)
    datetime_array = pd.to_datetime(existing_data_dict["date"], format="%Y%m%d")

# Get list of Excel files from folder
excel_files = glob(os.path.join(EXCEL_FOLDER, "*.xlsx"))

# Process each Excel file
# for file_path in tqdm(excel_files, desc="Processing Excel files"):

# file_path = excel_files[-1]
for file_path in excel_files:

    print(file_path)
    
    base_name = os.path.basename(file_path).split(".")[0]
    station, wellcode = base_name.split("_")
    
    # Get model groundwater level data
    model_gwl_arr = existing_data_dict[station][wellcode]["measure"]["model"]
    model_gwl_series = pd.Series(data=model_gwl_arr, index=datetime_array)
    
    # Read peaks and troughs data
    peaks_df = read_peak_trough_data(file_path, sheet_name="peaks", data_type="peaks")
    troughs_df = read_peak_trough_data(file_path, sheet_name="troughs", data_type="troughs")
    combined_df = pd.concat([peaks_df, troughs_df]).sort_values(by="dates").reset_index(drop=True)
    
    # Calculate thresholds for peaks and troughs
    peak_threshold = calculate_date_diff_threshold(
        peaks_df, date_col="dates", quantile=THRESHOLD_QUANTILE, min_threshold=MIN_THRESHOLD
    )
    trough_threshold = calculate_date_diff_threshold(
        troughs_df, date_col="dates", quantile=THRESHOLD_QUANTILE, min_threshold=MIN_THRESHOLD
    )
    
    # Identify indices to remove for peaks and troughs
    peak_indices_to_remove = identify_redundant_indices(
        peaks_df, threshold=peak_threshold, data_type="peaks", date_col="dates", value_col="values"
    )
    trough_indices_to_remove = identify_redundant_indices(
        troughs_df, threshold=trough_threshold, data_type="troughs", date_col="dates", value_col="values"
    )
    
    # Filter peaks and troughs DataFrames
    filtered_peaks_df = peaks_df.drop(peak_indices_to_remove).reset_index(drop=True)
    filtered_troughs_df = troughs_df.drop(trough_indices_to_remove).reset_index(drop=True)
    
    # Combine filtered peaks and troughs
    filtered_combined_df = pd.concat([filtered_peaks_df, filtered_troughs_df]).sort_values(by="dates").reset_index(drop=True)
    
    # Apply second filter to remove consecutive duplicates
    final_df = filter_consecutive_duplicates(filtered_combined_df, value_col="values", type_col="type")

    # ________________________________________________________________________________________________________________

    # Set the figure size and calculate the scaling factor
    fig_width, fig_height = visualize.BASE_SIZE
    scaling_factor = visualize.calculate_scaling_factor(fig_width, fig_height)
    
    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(fig_width * 1.5, fig_height * 2 / 3))
    
    # Plot the original data and smoothed data
    ax.plot(model_gwl_series, label="Modeled GWL", color="black", linewidth=1.5, alpha=0.8, zorder=1)
    
    # Plot peaks and troughs using scatter plots
    ax.scatter(
        combined_df.query("type=='peaks'")["dates"],
        combined_df.query("type=='peaks'")["values"],
        color="grey",
        marker="s",
        s=70,
        alpha=0.3,
        zorder=2,
    )
    ax.scatter(
        combined_df.query("type=='troughs'")["dates"],
        combined_df.query("type=='troughs'")["values"],
        color="grey",
        marker="s",
        s=70,
        alpha=0.3,
        zorder=2,
    )
    
    ax.scatter(
        final_df.query("type=='peaks'")["dates"],
        final_df.query("type=='peaks'")["values"],
        color="blue",
        marker="s",
        label="Peaks",
        s=70,
        alpha=0.7,
        zorder=3,
    )
    ax.scatter(
        final_df.query("type=='troughs'")["dates"],
        final_df.query("type=='troughs'")["values"],
        color="magenta",
        marker="s",
        label="Troughs",
        s=70,
        alpha=0.7,
        zorder=3,
    )
    
    # Annotate peaks with their corresponding indexes above the markers
    for i in range(len(peaks_df)):
        ax.text(
            peaks_df.loc[i, "dates"],
            peaks_df.loc[i, "values"] + 0.4,
            str(peaks_df.loc[i, "indexes"]),
            ha="center",
            va="bottom",
            rotation="vertical",
            fontsize=10 * scaling_factor,
        )
    
    # Annotate peaks with their corresponding indexes above the markers
    for i in range(len(troughs_df)):
        ax.text(
            troughs_df.loc[i, "dates"],
            troughs_df.loc[i, "values"] - 0.4,
            str(troughs_df.loc[i, "indexes"]),
            ha="center",
            va="top",
            rotation="vertical",
            fontsize=10 * scaling_factor,
        )
    
    
    # Configure the axis labels, title, and font scaling using visualize.py
    visualize.configure_axis(
        ax=ax, xlabel="Date", ylabel="Groundwater Levels (m)", title=f"{station} {wellcode}", scaling_factor=scaling_factor
    )
    
    # Configure the legend
    visualize.configure_legend(ax=ax, scaling_factor=0.5, frameon=False)
    
    # Configure ticks (datetime for x-axis)
    visualize.configure_datetime_ticks(ax=ax, axis="x", major_interval=12, minor_interval=6, date_format="%Y")
    
    # Adjust layout and set x-axis label rotation
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    plt.setp(ax.get_xticklabels(), rotation=90, ha="center")
    
    # Optional: Save the plot to a file (commented out by default)
    output_savename = f"{station}_{wellcode}"
    savepath = f"temp3_v2\\{output_savename}.png"
    visualize.save_figure(fig, savepath)
    
    # Close the plot to free memory (especially important in loops)
    plt.close()

temp4_v2\ANHE_10070111.xlsx
temp4_v2\ANHE_10070121.xlsx
temp4_v2\ANHE_10070131.xlsx
temp4_v2\ANHE_10070141.xlsx
temp4_v2\ANNAN_09140112.xlsx
temp4_v2\ANNAN_09140122.xlsx
temp4_v2\BEIGANG_09060112.xlsx
temp4_v2\BEIGANG_09060122.xlsx
temp4_v2\BOZI_09180111.xlsx
temp4_v2\BOZI_09180121.xlsx
temp4_v2\BOZI_09180131.xlsx
temp4_v2\CAICUO_09180211.xlsx
temp4_v2\CAICUO_09180221.xlsx
